# 現場で活用するためのAIエージェントワークショップ

[W&B Fully Connected Tokyo 2025](https://fullyconnected.jp/)

https://github.com/smiyawaki0820/wandb-fc-2025-agent-workshop

## 準備

- [x] [W&B API キーを発行](https://wandb.ai/authorize)
- [x] [OpenAI API キーを発行](https://platform.openai.com/api-keys)
- [x] [Perplexity API キーを発行](https://www.perplexity.ai/account/api)

※ 事前に配布したこちらを完了している想定で進めます🙇‍♂️<br/>
> https://colab.research.google.com/drive/1pMir7nu8twtc2Jq3G0HjeY-fKfBiHtE0

### git clone

In [3]:
%cd /content
! git clone https://github.com/smiyawaki0820/wandb-fc-2025-agent-workshop.git
%cd wandb-fc-2025-agent-workshop

/content
Cloning into 'wandb-fc-2025-agent-workshop'...
remote: Enumerating objects: 463, done.
remote: Counting objects: 100% (463/463), done.
remote: Compressing objects: 100% (241/241), done.
remote: Total 463 (delta 191), reused 457 (delta 191), pack-reused 0 (from 0)
Receiving objects: 100% (463/463), 286.28 KiB | 4.03 MiB/s, done.
Resolving deltas: 100% (191/191), done.
/content/wandb-fc-2025-agent-workshop


### 環境構築

In [9]:
! pip install -q -r requirements.txt

### 環境変数の設定

In [11]:
import os
from google.colab import userdata

os.environ["PROJECT_NAME"] = "wandb-ws-2025-ai-agent"
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["WANDB_API_KEY"] = userdata.get("WANDB_API_KEY")
os.environ["PERPLEXITY_API_KEY"] = userdata.get("PERPLEXITY_API_KEY")

# LangGraph

<figure>
  <img src="https://storage.googleapis.com/zenn-user-upload/1c2f8534f22d-20240624.png" width="30%" />
  <figcaption style="font-size: small;">PharmaXテックブログ「<a href="https://zenn.dev/pharmax/articles/8796b892eed183">LangGraphの基本的な使い方</a>」より引用</figcaption>
</figure>

In [12]:
from langchain_core.runnables import RunnableConfig
from langgraph.graph import StateGraph
from pydantic import BaseModel, Field

#### State

各ノードがアクセス可能な記憶領域

In [13]:
# State

class CheckStatus(BaseModel):
    is_accepted: bool
    reason: str

class State(BaseModel):
    user_message: str
    response_content: str | None = Field(default=None)
    check: CheckStatus | None = Field(default=None)

#### Node

特定の関数にあたる部分

In [14]:
def generate_response(state: State, config: RunnableConfig):
    return {"response_content": "Hello. I'm Taro."}

In [15]:
def check_response(state: State, config: RunnableConfig):
    return {"check": CheckStatus(is_accepted=True, reason="自然な回答です")}

#### Graph

各 Node と Edge（Node同士をつなぐ辺/遷移）の集合体

In [17]:
def create_workflow():
    graph_builder = StateGraph(State)
    # Graph にノードを追加
    graph_builder.add_node("generate_response", generate_response)
    graph_builder.add_node("check_response", check_response)
    # Nodeの関連をedgeに追加
    graph_builder.add_edge("generate_response", "check_response")
    # Graphの始点を宣言
    graph_builder.set_entry_point("generate_response")
    # Graphの終点を宣言
    graph_builder.set_finish_point("check_response")
    return graph_builder.compile()

#### グラフの実行

In [21]:
# Graphの実行
workflow = create_workflow()
for step in workflow.stream({"user_message": "Hello"}):
    print(step)

{'generate_response': {'response_content': "Hello. I'm Taro."}}
{'check_response': {'check': CheckStatus(is_accepted=True, reason='自然な回答です')}}


# DeepResearch

人間が行っていた情報収集、分析、レポート作成といった多段階の複雑な調査タスクをAIがエンドツーエンドに実行する。<br/>

特徴
- LLMの言語理解能力と推論能力を活用し、
- ユーザーの複雑な問いの意図を深く解釈し、
- それを複数の関連する調査タスクへと分解した後、
- 膨大な情報源から、自律的に情報を収集、分析、そして統合する。

参考
- [OpenAI DeepResearch](https://openai.com/ja-JP/index/introducing-deep-research/)
- [Gemini DeepResearch](https://gemini.google/overview/deep-research/)
- [Perplexity DeepResearch](https://www.perplexity.ai/ja/hub/blog/introducing-perplexity-deep-research)
- [github.com/langchain-ai/openai_deep_research](https://github.com/langchain-ai/open_deep_research/tree/main)
- [github.com/SalesforceAIResearch/enterprise-deep-research](https://github.com/SalesforceAIResearch/enterprise-deep-research)

Q. DeepResearch API があるのになぜ一から作成するの？<br/>
- 本ワークショップでは DeepResearch を構築するのが目的ではありません
- 知見として持ち帰ってもらえそうなところをピックアップしており、その一つの教材として DeepResearch を採用しました


## 🚩 Step0. まずは動かしてみる

### 全体像

<img src="https://i.gyazo.com/a20ed93b5947cd134e03acdcd874a269.png" />

### クイックスタート

In [ ]:
%cd /content/wandb-fc-2025-agent-workshop
! python main.py -m "AIエージェントの登場によりBPOが注目されるようになっていますが、今後注目される領域やビジネスモデルはどのようなものがあると考えられますか？"

/content/wandb-fc-2025-agent-workshop
2025-10-29 20:03:14.723 | DEBUG    | app.core.logging:log:26 - [GatherRequirementsNode] invoke | inquiry_items_evaluation=[] additional_questions=[AdditionalQuestion(question='この先のBPOで注目される領域について、具体的に関心がある分野を教えてください（例: アウトソーシングの新しいビジネスモデル、AI統合、データ分析、品質保証など）', priority=<Priority.HIGH: 'high'>), AdditionalQuestion(question='検討しているビジネスモデルの種類は何ですか？例えば、成果報酬型、サブスクリプション、ハイブリッドモデル、オンデマンド型、パフォーマンス連動型など。', priority=<Priority.HIGH: 'high'>), AdditionalQuestion(question='対象となる地域・市場はどこですか（例: 日本、英語圏、グローバル）', priority=<Priority.MEDIUM: 'medium'>), AdditionalQuestion(question='調査の期間はどれくらいを想定しますか？過去データ・将来の展望のどちらを重視しますか', priority=<Priority.LOW: 'low'>)] skip_gather_requirements=False inquiry_items=[ManagedInquiryItem(id='0st_3', status=<ManagedTaskStatus.NOT_STARTED: 'not_started'>, answer=None, priority=<Priority.HIGH: 'high'>, question='この先のBPOで注目される領域について、具体的に関心がある分野を教えてください（例: アウトソーシングの新しいビジネスモデル、AI統合、データ分析、品質保証など）'), ManagedInquiryItem(id='rQItP', status=<Mana

In [ ]:
! cat storage/outputs/research_report.md

以下は、ご提供のストーリーラインに完全準拠して作成した「レポート生成のための最終アウトプット設計案（Markdown版テンプレート兼実装指針）」です。現時点ではドラフトの設計・構成案・データ要件・アウトプット様式を含むため、実データが揃い次第、数値・ケーススタディ・実証データを埋め込み、引用付きの正式版へと更新します。データソースの確定後には、各主張を明確な出典付きで裏付け、章ごとの論理展開を強化します。

使い方の要点
- 本案は「ストーリーライン」を忠実に反映した、並行実行可能なタスク設計と、出力フォーマットの完全版テンプレートです。
- すべての主張は、適切な引用を付して裏付ける前提で進めます。正式版では、各データポイントに対して出典を明記します。
- 表・図はMarkdown形式で表現可能なものを用意しています。データ未確定の箇所はプレースホルダを置き、データ取得後に更新します。
- Web検索データの組み込みは、APIキーの問題解消後に実行可能です。現状はドラフト段階として、データ収集計画と出力フォーマットを整えています。

1. レポート全体の構成と狙い（ストーリーラインに沿った章立て）

- 表紙・要約
  - 目的、対象市場、日本市場における将来展望の要点、主要提案のエッセンスを1ページ要約として提示
- 1. はじめに
  - 本計画の目的と前提、ユーザー要求の再定義、提案するレポートの位置づけと期待成果
- 2. 背景と動機
  - 日本市場のBPOの現状、AI統合・データ活用・品質保証領域の成長ポテンシャル、国内固有要因の整理
- 3. 調査目的と設計方針
  - 主な問いと現実的設計方針、潜在要望の補完方針、並行タスク設計、再現性確保の基準
- 4. 調査対象・前提条件
  - 日本市場、将来展望、3つのビジネスモデル中心の設定、期間・セグメント区分
- 5. 研究手法とデータソース
  - 文献調査、産業レポート、政府統計、企業公開情報、データ取り扱い・再現性の確保方法
- 6. 日本市場の現状と将来展望
  - 市場規模・成長性・動向の現状仮説と将来仮説、セグメント別動向の概観
- 7. 新しいアウトソーシングビジネスモデルの比較
  - 成果報酬型、サブスクリプション型、ハイブリッドの価値提案・費用構造・リスク・適用条件の比較表
- 8

## 🚩 Step1. 調査要件を整理する

ユーザーの要求を正確に把握することは全ての LLM アプリケーションにおいて重要です。

ユーザー要求の把握（ヒアリング）には、いくつかの実現手段が考えられます。
- ヒアリングシートのような事前に作成された質問に回答してもらう
- 対話を通じて要求事項をリストアップする
- etc...

今回は調査スコープを制限しないため **「対話を通じて要求事項をリストアップする」** 枠組みを採用します。

<img src="https://i.gyazo.com/f87db96cd79c7880b0b885c313bd8cdf.png" />

### ▶️ Gather Requirements / 要求の収集

- ユーザーとの対話履歴を受け取り、ヒアリングシートを更新する
- タスクを遂行する上で不足情報があればユーザーに提示する追加質問を作成する
- 不足情報がなければタスク計画に移る


#### 🤖 ノード

In [38]:
from typing import Literal

from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.types import Command

from app.core.logging import LogLevel
from app.core.utils.nano_id import generate_id
from app.domain.enums import ManagedTaskStatus, Priority
from app.infrastructure.blob_manager import BaseBlobManager
from app.infrastructure.llm_chain.openai_chain import BaseOpenAIChain
from app.infrastructure.llm_chain.enums import OpenAIModelName
from app.workflow.enums import Node
from app.workflow.models import (
    GatherRequirements,
    ManagedInquiryItem,
    ResearchAgentState,
)


In [33]:
class GatherRequirementsNode(BaseOpenAIChain):
    def __init__(
        self,
        model_name: OpenAIModelName,
        blob_manager: BaseBlobManager,
        log_level: LogLevel = LogLevel.DEBUG,
        prompt_path: str = "storage/prompts/research_agent/nodes/gather_requirements.jinja",
    ) -> None:
        super().__init__(model_name, blob_manager, log_level, prompt_path)

    def __call__(
        self,
        state: ResearchAgentState
    ) -> Command[Literal[Node.FEEDBACK_REQUIREMENTS.value, Node.BUILD_RESEARCH_PLAN.value]]:
        gather_requirements = self.run(state.messages, state.inquiry_items)
        # 既存の要件収集項目のステータスを更新
        state.inquiry_items = gather_requirements.update_inquiry_items(
            state.inquiry_items
        )
        # 新しい要件収集項目を追加
        state.inquiry_items += gather_requirements.inquiry_items
        return Command(
            # LLM が「タスク要求を十分に収集できた」と判断した場合（is_completed=True）タスク計画の立案に進み、そうでない場合は要求への回答に進む
            goto=(
                Node.BUILD_RESEARCH_PLAN.value
                if gather_requirements.is_completed
                else Node.FEEDBACK_REQUIREMENTS.value
            ),
            update=state,
        )

    def run(
        self,
        messages: list[BaseMessage],
        managed_inquiry_items: list[ManagedInquiryItem],
        verbose: bool = False,
    ) -> GatherRequirements:
        chain = self._build_structured_chain(GatherRequirements)
        inputs = {
            "conversation_history": messages,
            "managed_inquiry_items": managed_inquiry_items,
            "output_format": GatherRequirements.model_json_schema(),
        }
        return self.invoke(chain, inputs, verbose)


#### 動かしながら説明

##### 📝 入力データ

1. 対話履歴: `list[BaseMessage]`
2. 質問リスト: `list[ManagedInquiryItem]`

In [34]:
# [入力データ] 対話履歴

# 以下のような対話履歴を考えます。

messages = [
    # t=0
    HumanMessage(content="Transformersの論文を探しています。"),
    # t=1
    AIMessage(content="検索対象とする期間（例：最近の年、過去10年、特定の期間など）を教えてください。"),
    HumanMessage(content="最近の年でお願いします。"),
    # t=2
    AIMessage(content="どのような分野や用途でお探しですか？（例：自然言語処理、画像認識、医療データ分析など）"),
    HumanMessage(content="医療データの解析に活用したいと考えています。"),
]

In [39]:
# [入力データ] 質問リスト

# 各質問は `ManagedInquiryItem` としてステータスが管理されます。
# ここでは対話が3ターン目の状態で、以下の質問リストを保持していることを考えます。

state_inquiry_items = [
    # t=1 で入力された質問（ステータス更新済）
    ManagedInquiryItem(
        id=generate_id(),
        status=ManagedTaskStatus.COMPLETED.value,
        question="検索対象とする期間（例：最近の年、過去10年、特定の期間など）を教えてください。",
        answer="最近の年",
        priority=Priority.MEDIUM.value
    ),
    # t=2 で入力された質問（ステータス未更新）
    ManagedInquiryItem(
        id=generate_id(),
        status=ManagedTaskStatus.NOT_STARTED.value,
        question="どのような分野や用途でお探しですか？（例：自然言語処理、画像認識、医療データ分析など）",
        answer=None,
        priority=Priority.HIGH.value
    ),
]

##### 🏃‍♀️ 実行してみる

In [40]:
from langchain_core.messages import HumanMessage, AIMessage

from app.domain.enums import ManagedTaskStatus, Priority
from app.infrastructure.blob_manager import LocalBlobManager

blob_manager = LocalBlobManager()
chain = GatherRequirementsNode(OpenAIModelName.GPT_5_MINI, blob_manager)

In [41]:
# [実行] GatherRequirements ノード
gather_requirements = chain.run(messages, state_inquiry_items, verbose=False)

# [実行] state の更新
state_inquiry_items = gather_requirements.update_inquiry_items(state_inquiry_items)
state_inquiry_items += gather_requirements.inquiry_items

2025-10-30 00:50:55.768 | DEBUG    | app.core.logging:log:26 - [GatherRequirementsNode] invoke | inquiry_items_evaluation=[ManagedItem(id='YyJbe', status=<ManagedTaskStatus.COMPLETED: 'completed'>, answer='最近の年'), ManagedItem(id='uH_Z3', status=<ManagedTaskStatus.COMPLETED: 'completed'>, answer='医療データの解析')] additional_questions=[AdditionalQuestion(question='最近の“最近の年”は何年間を想定しますか？（例：過去1年、過去3年、過去5年）', priority=<Priority.HIGH: 'high'>), AdditionalQuestion(question='医療データの種類は何ですか？例：電子カルテ（臨床テキスト）、医用画像（CT/MRI/X-ray）、ゲノム・オミクスデータ、生体信号（心電図など）など。該当するものを選んでください。', priority=<Priority.HIGH: 'high'>), AdditionalQuestion(question='成果物の形式と範囲：必要な論文数（例：10本、20本）、要約の深さ（短い要約／手法と結果の詳細含む）、査読済みのみかarXiv等のプレプリントも含めるか、検索言語（英語／日本語）を教えてください。', priority=<Priority.MEDIUM: 'medium'>)] skip_gather_requirements=False inquiry_items=[ManagedInquiryItem(id='Y1K5D', status=<ManagedTaskStatus.NOT_STARTED: 'not_started'>, answer=None, priority=<Priority.HIGH: 'high'>, question='最近の“最近の年”は何年間を想定しますか？（例：過去1年、過去3年、過去5年）'), Manag

##### 👀 出力結果の確認

In [42]:
# [確認] 更新結果
#   1. `state_inquiry_items[1].id` に該当する質問事項の {status, answer} が更新されていることを確認
#   2. `state_inquiry_items` に加えて、ユーザーに追加で聞きたい質問事項が増えていることを確認

for updated_item in state_inquiry_items:
    print(updated_item.model_dump_json())

{"id":"YyJbe","status":"completed","answer":"最近の年","priority":"medium","question":"検索対象とする期間（例：最近の年、過去10年、特定の期間など）を教えてください。"}
{"id":"uH_Z3","status":"completed","answer":"医療データの解析","priority":"high","question":"どのような分野や用途でお探しですか？（例：自然言語処理、画像認識、医療データ分析など）"}
{"id":"B7itU","status":"not_started","answer":null,"priority":"high","question":"最近の“最近の年”は何年間を想定しますか？（例：過去1年、過去3年、過去5年）"}
{"id":"BeZRB","status":"not_started","answer":null,"priority":"high","question":"医療データの種類は何ですか？例：電子カルテ（臨床テキスト）、医用画像（CT/MRI/X-ray）、ゲノム・オミクスデータ、生体信号（心電図など）など。該当するものを選んでください。"}
{"id":"i6q0e","status":"not_started","answer":null,"priority":"medium","question":"成果物の形式と範囲：必要な論文数（例：10本、20本）、要約の深さ（短い要約／手法と結果の詳細含む）、査読済みのみかarXiv等のプレプリントも含めるか、検索言語（英語／日本語）を教えてください。"}


- ユーザーとの対話履歴を受け取り、ヒアリングシートを更新する
- タスクを遂行する上で不足情報があればユーザーに提示する追加質問を作成する

##### 📝 プロンプト

In [152]:
! cat storage/prompts/research_agent/nodes/gather_requirements.jinja

---
{{ global_instruction }}
---

情報調査のプロフェッショナルであるあなたに「ユーザーヒアリング」に関する依頼です。
ユーザー要求が入力データとして与えられるので、情報調査をしたいと思う背景や、ユーザーが解決したい課題を正確に把握して下さい。
ユーザーの要望に対して最適な調査結果を提供するために、検索対象や検索条件を明確にすることです。

ユーザーは自身の要求に対して「どのように目的を達成すれば良いか」自己解決するための道筋を言語化できていない可能性があります。
ユーザーとのヒアリングを通してユーザーの課題を明らかにし、調査対象の検索クエリとなるヒアリングシートを構築して下さい。


## ヒアリングの実施手順

1. ユーザーの初期クエリを注意深く分析し、不明確な点や追加情報が必要な箇所を特定する。
2. 情報不足により適切な調査ができないと判断される場合は、最適な情報を引き出すための質問集を作成する。
3. 会話履歴を参照し、既存の質問事項に対する回答をユーザーから取得済みである場合は、その質問事項のステータスを更新する。

### ヒアリングの完了基準

以下のいずれかに該当する場合にヒアリングを終了とし、出力項目である `skip_gather_requirements` を True とする。
また同時に TaskStatus を `pending` に更新する。

- 最適な情報調査を実施するために必要な情報を十分に得られている場合
- 追加情報を求める質問にユーザーが意図的に回答しなかった場合（明示的に不明や未定という回答が得られた場合）
- 質問に対してユーザーから「NO ANSWER NEEDED」などの回答を得た場合。
- ユーザーから「そのままの条件で検索し、調査してください。」「/skip」「これ以上の要件収集は不要です」というような、ショートカット回答を得た場合


## 出力制約

- 情報調査にあたり、対象となる分野や期間などを確認し、情報源のトピックを絞り込む。
- 優先度が高い場合に、追加の詳細や状況説明をユーザーに求める。過去の対話における質問数を含めて、ヒアリング項目合計数が5を超えると離脱率が高くなる傾向があるため、質問の優先度を常に考慮すること。
- 質問をする際は、ユーザーが専門用語や高度

### ▶️ Feedback Requirements / 要求に関するユーザ回答


- 作成された質問をユーザーに提示して回答を求める

<img src="https://i.gyazo.com/f87db96cd79c7880b0b885c313bd8cdf.png" />

##### コードの説明だけします！

内部の動作としては以下のような流れになります。

1. `[FeedbackRequirementsNode]` state を受け取って実行開始
2. `[FeedbackRequirementsNode]` ワークフローの実行を一時中断（interrupt）してユーザーに回答の入力を求める
3. `[main]` interrupt 信号を検知
4. `[main]` ユーザーに質問リストを提示して回答を要求
5. `[main]` 回答データを FeedbackRequirementsNode に返す
6. `[FeedbackRequirementsNode]` 回答が返ってきたら state を更新

In [43]:
from typing import Literal

from langchain_core.messages import AIMessage, HumanMessage
from langgraph.types import Command, interrupt

from app.workflow.models import ResearchAgentState
from app.core.logging import LogLevel
from app.infrastructure.llm_chain import BaseChain
from app.workflow.enums import Node


class FeedbackRequirementsNode(BaseChain):
    def __call__(self, state: ResearchAgentState) -> Command[Literal[Node.GATHER_REQUIREMENTS.value]]:
        self.log(object="feedback_requirements", message=f"state: {state}")
        # ワークフローの実行を一時中断（interrupt）、ユーザーに回答の入力を要求
        feedback_items = interrupt({
            "node": self.__name__,
            "inquiry_items": state.inquiry_items,
        })
        # ユーザーから回答が返ってきたらIDで照合された質問のステータスを更新
        for idx, previous_item in enumerate(state.inquiry_items):
            if current_item := feedback_items.get(previous_item.id):
                state.inquiry_items[idx] = current_item
                # 対話履歴に Q&A （質問とユーザーからの回答）を追加
                state.messages.extend([
                    AIMessage(content=current_item.question),
                    HumanMessage(content=current_item.answer),
                ])
        return Command(goto=Node.GATHER_REQUIREMENTS.value, update=state)

#### 実行サイドの interrupt の記述例

```python
# main.py より抜粋

def invoke_graph(
    graph: CompiledStateGraph,
    input_data: dict | Command,
    config: dict,
) -> dict:
    result = graph.invoke(
        input=input_data,
        config=config,
    )
    for interrupt in result.get("__interrupt__", []):
        if interrupt_data := getattr(interrupt, "value", None):
            match interrupt_data.get("node"):
                case Node.FEEDBACK_REQUIREMENTS.value:
                    inquiry_items = deepcopy(interrupt_data.get("inquiry_items", []))
                    # 質問リスト（inquiry_items）をイテレーション
                    for idx, inquiry_item in enumerate(inquiry_items):
                        # 未回答の質問のみを対象に、ユーザーに回答を求める
                        if inquiry_item.status in [ManagedTaskStatus.NOT_STARTED]:
                            question = inquiry_item.question
                            user_input = str(input(f"{question} > "))
                            # 回答を得たら、answer, status をそれぞれ更新する
                            inquiry_items[idx].answer = user_input or "NO ANSWER NEEDED"
                            inquiry_items[idx].status = ManagedTaskStatus.COMPLETED

                    # Command(resume=...) で、ワークフローを再開する
                    return invoke_graph(
                        graph=graph,
                        input_data=Command(
                            # ユーザーからの回答データを {ID: 回答} の形式で FeedbackRequirementsNode に返す
                            resume={item.id: item for item in inquiry_items}
                        ),
                        config=config,
                    )
                case _:
                    error_message = f"Unknown node: {interrupt_data.get('node')}"
                    raise ValueError(error_message)
    return result
```


### ▶️ Build Research Plan / タスク計画の立案

- ヒアリングシートをもとにレポートの作成計画を立案する
- 各計画の実行をAIエージェントに依頼する

<img src="https://i.gyazo.com/f87db96cd79c7880b0b885c313bd8cdf.png" />

※ StructuredOutput を用いてタスク計画を立案しているだけなので詳細は割愛します

In [49]:
from typing import Literal

from langchain_core.messages import BaseMessage
from langgraph.types import Command, Send

from app.core.logging import LogLevel
from app.domain.enums import Priority
from app.infrastructure.blob_manager import BaseBlobManager
from app.infrastructure.llm_chain.openai_chain import BaseOpenAIChain
from app.infrastructure.llm_chain.enums import OpenAIModelName
from app.workflow.enums import Node
from app.workflow.models import (
    ResearchAgentState,
    ResearchPlan,
    ManagedInquiryItem,
    ExecuteTaskState,
)
from app.workflow.nodes.build_research_plan import BuildResearchPlanNode

#### 動かしながら説明

##### 📝 入力データ

1. 対話履歴: `list[BaseMessage]`
2. 質問リスト: `list[ManagedInquiryItem]`

In [50]:
# [入力データ] 対話履歴

messages = [
    # t=0
    HumanMessage(content="Transformersの論文を探しています。"),
    # t=1
    AIMessage(content="検索対象とする期間（例：最近の年、過去10年、特定の期間など）を教えてください。"),
    HumanMessage(content="最近の年でお願いします。"),
    # t=2
    AIMessage(content="どのような分野や用途でお探しですか？（例：自然言語処理、画像認識、医療データ分析など）"),
    HumanMessage(content="医療データの解析に活用したいと考えています。"),
]

In [51]:
# [入力データ] 質問リスト

state_inquiry_items = [
    # t=1 で入力された質問（ステータス更新済）
    ManagedInquiryItem(
        id=generate_id(),
        status=ManagedTaskStatus.COMPLETED.value,
        question="検索対象とする期間（例：最近の年、過去10年、特定の期間など）を教えてください。",
        answer="最近の年",
        priority=Priority.MEDIUM.value
    ),
    # t=2 で入力された質問（ステータス更新済）
    ManagedInquiryItem(
        id=generate_id(),
        status=ManagedTaskStatus.COMPLETED.value,
        question="どのような分野や用途でお探しですか？（例：自然言語処理、画像認識、医療データ分析など）",
        answer="医療データの解析",
        priority=Priority.HIGH.value
    ),
]

##### 🏃‍♀️ 実行してみる

In [53]:
blob_manager = LocalBlobManager()
plan_node = BuildResearchPlanNode(OpenAIModelName.GPT_5_NANO, blob_manager)

In [55]:
# [実行] 調査計画の立案
research_plan = plan_node.run(
    messages=messages,
    inquiry_items=state_inquiry_items,
    verbose=False
)

2025-10-30 00:56:58.772 | DEBUG    | app.core.logging:log:26 - [BuildResearchPlanNode] invoke | goal='直近の年に公開されたTransformer関連論文の中から、医療データ解析への応用を中心に網羅的に調査し、実務・研究設計に直結する要点をわかりやすく整理・比較・評価するレポートを作成する。ユーザーが具体的な用途・研究計画を立てられるよう、領域横断の主要手法とデータモダリティ、エビデンスの信頼性、倫理・規制上の留意点を統合して提示する。' acceptance_criteria='- 直近1年に発表された論文を対象とする。\n- 医療データ解析への適用が明示されていること。\n- 複数のデータモダリティ（NLP/EHR、医用画像、時系列データ等）を網羅。\n- モデル種別（BERT系、GPT系、Vision Transformer、Time-series Transformer など）の医療分野適用を比較。\n- 各論文の品質評価（サンプルサイズ、検証、外部検証の有無）を行い、エビデンスの強さを記述。\n- 推奨事項と限界、臨床導入の課題を明記。\n- レポート構成とアウトラインを提示。\n- 参照・検索戦略・データ抽出シート・再現性のための付録を提供。' storyline=[ReportSection(section='はじめに', description='本リサーチの背景・目的を明確にし、ユーザーの潜在ニーズ（医学データ解析へTransformerを適用する際の実務的指針、研究計画の設計、信頼性の高い要約）を仮説として提示する。'), ReportSection(section='対象と選定基準', description='対象期間は直近1年、医療データ解析に関する論文を絞り込み。言語は英語論文を中心に、非医療領域や非Transformer系技術は除外。選定基準（データモダリティ、モデル種別、評価設計、外部検証の有無等）を定義する。'), ReportSection(section='研究方法と手順', description='検索データベース（PubMed、arXiv、bioRxiv/medRxiv、IEEE Xplore、ACMなど）と検索語（例: T

##### 👀 出力結果の確認

In [56]:
# [確認] 調査計画
print(research_plan.model_dump_json(indent=2))

{
  "goal": "直近の年に公開されたTransformer関連論文の中から、医療データ解析への応用を中心に網羅的に調査し、実務・研究設計に直結する要点をわかりやすく整理・比較・評価するレポートを作成する。ユーザーが具体的な用途・研究計画を立てられるよう、領域横断の主要手法とデータモダリティ、エビデンスの信頼性、倫理・規制上の留意点を統合して提示する。",
  "acceptance_criteria": "- 直近1年に発表された論文を対象とする。\n- 医療データ解析への適用が明示されていること。\n- 複数のデータモダリティ（NLP/EHR、医用画像、時系列データ等）を網羅。\n- モデル種別（BERT系、GPT系、Vision Transformer、Time-series Transformer など）の医療分野適用を比較。\n- 各論文の品質評価（サンプルサイズ、検証、外部検証の有無）を行い、エビデンスの強さを記述。\n- 推奨事項と限界、臨床導入の課題を明記。\n- レポート構成とアウトラインを提示。\n- 参照・検索戦略・データ抽出シート・再現性のための付録を提供。",
  "storyline": [
    {
      "section": "はじめに",
      "description": "本リサーチの背景・目的を明確にし、ユーザーの潜在ニーズ（医学データ解析へTransformerを適用する際の実務的指針、研究計画の設計、信頼性の高い要約）を仮説として提示する。"
    },
    {
      "section": "対象と選定基準",
      "description": "対象期間は直近1年、医療データ解析に関する論文を絞り込み。言語は英語論文を中心に、非医療領域や非Transformer系技術は除外。選定基準（データモダリティ、モデル種別、評価設計、外部検証の有無等）を定義する。"
    },
    {
      "section": "研究方法と手順",
      "description": "検索データベース（PubMed、arXiv、bioRxiv/medRxiv、IEEE Xplore、ACMなど）と検索語（例: Transformer, BERT, GPT, Vision Transf

生成された `managed_tasks` を次の ExecuteTaskNode に渡すことでタスク実行を並行して行います。

##### 📝 プロンプト

In [52]:
! cat storage/prompts/research_agent/nodes/build_research_plan.jinja

---
{{ global_instruction }}
---

情報調査のプロフェッショナルであるあなたに「リサーチ計画の立案」に関する依頼です。
ユーザーからのヒアリング結果と会話履歴が入力データとして与えられるので、リサーチとレポートを成功させるための情報調査の手順を明確にして下さい。
計画立案の目的は、ユーザーの要望に対して最適な調査結果を提供するために、現実的な道筋を立てることです。

ユーザーは自身の要求を適切に言語化をできない可能性があります。
ユーザー要求やヒアリング結果については、ユーザーの潜在要求を推測して、適切な形で補完すること。
ユーザーからの回答結果を元に調査目的を再定義し、最適な調査結果を提供するための計画を立案して下さい。

## リサーチ計画立案の実施手順

1. ユーザーからのヒアリング結果と会話履歴を注意深く分析し、調査目的とレポートイメージを再定義する。
2. 調査目的とレポートイメージから逆算し、レポートに含めるべき調査対象と方針を明確にする。
3. 調査対象と方針を考案したら、最適な調査結果を提供する目的を達成するために、リサーチ計画としてタスクばらしを行う。


## 出力制約

- リサーチ計画の対象言語は日本語とする。
- リサーチ計画としてのサブタスク系列は並行処理可能な互いに独立した単位で分解し、時系列や因果関係が重要な場合でも、他のタスクの結果に依存しない。
- 高いカバレッジおよび深い理解を達成すべく、「〜について調査する」など、計画対象は特定のテーマに関する調査のみに限定する。
- 調査対象は、自然言語で記述されたコンテンツとする（メタデータやbib、コードについては調査対象から除外する）。
- ヒアリング結果を参照する際は、優先度（priority）とステータス（status）を確認し、優先度と実現可能性の高い項目から計画に反映する。
- ユーザーの要望を確実に満たすために必要な記述内容は、第三者が再現できるよう具体的かつ明確な表現を心がける。
- 包括的な調査を実施する場合、まとめ・サーベイ記事を対象とし、全体像を把握できる信頼性の高い情報源を優先的に参照する。
- 計画としてのサブタスクには、ユーザーに依存する作業を含めない（追加ヒアリングなど）
- 専門用語や概念は、誤解のないよう正確かつ丁寧に記述

## 🚩 Step2. タスクの実行


ここでは LangChain の [`create_agent`](https://docs.langchain.com/oss/python/releases/langchain-v1#create-agent) を使用します。<br/>

エージェントは ReAct (「推論 + 行動」) パターンに従い、短い推論ステップと対象を絞ったツール呼び出しを交互に実行し、結果として得られた観察結果を後続の決定にフィードバックして、最終的な答えを出すまで続けます。

<img src="https://i.gyazo.com/bb7a30ca6303a671564328f8f9b83c43.png" width="" />

### 🪛 ツール

#### ① search_web / Web検索

- Perplexity の [Search API](https://docs.perplexity.ai/guides/search-guide#response) を使用して Web から関連するページ情報を取得

In [57]:
import json

from perplexity.types.search_create_response import SearchCreateResponse
from perplexity import Perplexity


def search_web(search_view: str) -> str:
    """指定されたキーワードでWeb検索を行い、検索結果を返します.

    Args:
        search_view (str): Web検索に使用するキーワードや観点です。例えば、特定の技術や最新動向に関するキーワードを指定することで、より広範囲かつ深い情報収集が可能になります。

    Returns:
        SearchCreateResponse: 検索結果。
    """
    client = Perplexity()
    search_create_response: SearchCreateResponse = client.search.create(
        query=search_view, max_results=3, max_tokens_per_page=512
    )
    return json.dumps(
        [
            {
                "title": result.title,
                "url": result.url,
                "snippet": result.snippet,
            }
            for result in search_create_response.results
        ],
        ensure_ascii=False,
    )

In [60]:
response_content = search_web("W&B の Weave について教えて")
for result in json.loads(response_content):
    print(result)

{'title': 'W&B Weave\u200b', 'url': 'https://wandb.ai/site/weave/', 'snippet': '# Deliver AI with confidence\n\n```\n\nimport weave\n\nweave.init("quickstart")\n\n@weave.op()\n\ndef llm_app(prompt):\n\n```\n\n## Improve quality, cost, latency, and safety\n\nWeave works with any LLM and framework and comes with a ton of integrations out of the box\n\n### Quality\n\nAccuracy, robustness, relevancy\n\n### Cost\n\nToken usage and estimated cost\n\n### Latency\n\nTrack response times and bottlenecks\n\n### Safety\n\nProtect your end users using guardrails\n\n## Measure and iterate\n\n### Visual comparisons\n\nUse powerful visualizations for objective, precise comparisons\n\n### Automatic versioning\n\nSave versions of your datasets, code, and scorers\n\n```\n\nimport openai, weave\n\nweave.init("weave-intro")\n\n@weave.op\n\ndef correct_grammar(user_input):\n\nclient = openai.OpenAI()\n\nresponse = client.chat.completions.create(\n\nmodel="o1-mini",\n\nmessages=[{\n\n"role": "user",\n\n"con

#### ② submit_content / 中間生成物の確認


- AIエージェントがいい感じに調査結果をまとめられたと判断したら呼び出す
- 調査結果に対して受理可能かどうかを判定する

In [61]:
from typing import cast, TYPE_CHECKING

from dotenv import load_dotenv
from langchain_core.tools import tool
from openai import OpenAI
from openai.types.responses.parsed_response import ParsedResponse
from pydantic import BaseModel, Field, computed_field

from app.core.logging import LogLevel
from app.domain.enums import BaseEnum
from app.infrastructure.blob_manager import LocalBlobManager

In [62]:
# 出力形式

class EvaluationStatus(BaseEnum):
    ACCEPTED = "accepted"
    IMPROVABLE = "improvable"
    REJECTED = "rejected"


class Submission(BaseModel):
    status: EvaluationStatus = Field(
        title="提出物の評価結果",
        description=(
            "提出物の評価は3つのパターンで分けます。"
            "1. 'accepted'（受理）: 要件・期待値をすべて満たしており受け入れ可能。"
            "2. 'improvable'（改善の余地あり）: いくつか要件は満たしているが改善すべき点があり修正を推奨。"
            "3. 'rejected'（終了）: 主な要件が満たされておらず今回は受理不可。"
            "このいずれかを厳密に記入してください。"
        ),
    )
    reason: str = Field(
        title="受け入れ判断の詳細理由",
        description=(
            "提出物が受け入れられなかった場合は、どこが要件や期待に達していなかったのか、具体的な改善ポイントやアドバイスを丁寧に記述してください。"
            "逆に、受理された場合は、どの点が要件を満たしていたのか、なぜ十分だったのかについて積極的なフィードバックや称賛を含めて説明してください。"
        ),
        exclude=True,
    )

    @computed_field
    @property
    def reason_for_rejection(self) -> str | None:
        return None if self.status in [EvaluationStatus.REJECTED] else self.reason

In [63]:
def submit_content(content: str) -> dict[str, str | bool | None]:
    """指定された提出物の内容を審査し、受け入れ可否および理由を返します。
    提出物はマークダウン形式で記述してください。本関数は、その内容が所定の要件や期待を満たしているかどうかを判定し、
    判定結果（受け入れ可否）および詳細な理由や改善点（または称賛ポイント）を返します。.

    Args:
        content (str): マークダウン形式で記述された確認対象の提出物の内容。

    Returns:
        Submission:
            is_accepted（bool）: 提出物が受け入れ可能かどうか（True: 受け入れ／False: 不可）。
            reason（str）: 受け入れ可否の根拠や詳細な理由。非受理の場合は改善ポイント、受理の場合は称賛コメントなどを含みます。
    """
    client = OpenAI()
    blob_manager = LocalBlobManager(log_level=LogLevel.TRACE)
    system_instruction_template = blob_manager.read_blob_as_template(
        "storage/prompts/research_agent/tools/submit_content.jinja"
    )
    system_instruction = system_instruction_template.render(
        output_format=Submission.model_json_schema()
    )
    result: ParsedResponse[Submission] = client.responses.parse(
        model="gpt-5-nano",
        input=[
            {"role": "system", "content": system_instruction},
            {"role": "user", "content": content},
        ],
        text_format=Submission,
        reasoning={"effort": "low"},
        text={"verbosity": "low"},
    )
    submission: Submission = cast("Submission", result.output_parsed)
    return submission.model_dump()

In [64]:
# 試しに submit_content を実行してみる

submission = submit_content("hoge")
submission

{'status': <EvaluationStatus.REJECTED: 'rejected'>,
 'reason_for_rejection': None}

In [65]:
# 試しに submit_content を実行してみる

content = open("storage/outputs/research_report.md").read()
submission = submit_content(content)
submission

{'status': <EvaluationStatus.IMPROVABLE: 'improvable'>,
 'reason_for_rejection': '総論としては、日本市場のHR/BPO領域を網羅的に整理した初稿ドラフトとして高品質です。特に全体像・前提・トピックの整理、ケース型の案内、今後の深掘り課題とロードマップ案の骨格は整っています。しかし、提出要件を満たすために、いくつか重要な改善ポイントがあります。\n\n改善点と具体的な対応案\n- 明確性と構造の強化\n  - 目次・セクション番号を付与し、各章の目的と要点を冒頭に箇条書きで示すと読み手が迅速に理解できます。\n  - 主要セクションの「結論/要点」と「根拠となる情報・出典」を分離し、読了後の要点把握を容易にする。\n- 出典とデータの透明性\n  - 「出典・参考リンク」がケース別・定量的主張に対して明示されていない箇所があるため、定量データ（市場規模、CAGR、KPI指標の標準値など）については、出典を具体的に明示してください。\n  - 事例A–Fの教訓・背景情報は羅列ベースになっているため、各ケースに紐づく出典・実績指標を明確化（例：ケースAはどの出典のどの要素に根拠があるか）する。\n- ケース別マトリクス表と5年間ロードマップの具体性\n  - 「ケース別マトリクス表（A–F）」と「5年間ロードマップ案」を別ファイル/別セクションとして出力可能との案は良いですが、それぞれのテンプレ案が本文中で十分に定義されていません。各ケースの要件、適用領域、代表KPI、導入難易度、ROIの目安、失敗パターンを具体化したテンプレを用意してください。\n  - ロードマップ案について、Yearごとの具体的なタスク、責任者、依存関係、リスク、KPIのトラッキング方法を明記する必要があります。\n- 要件適合の確認ポイントの追加\n  - ケースA–Fが満たすべき要件（例：SLA設定、データ統合要件、法規制遵守、セキュリティ要件、ガバナンス体制、データ品質指標、組織設計の柔軟性）を列挙し、それぞれのケースがどう対応するかをマッピングしてください。\n- 実務性とユースケースの拡充\n  - SME向けROIの検証計画、グローバル展開時の法規制・データ主権の考慮、具体的なKPIダッシュボード例（画面構

### 📝 入力プロンプト

In [66]:
blob_manager = LocalBlobManager()

prompt_path = "storage/prompts/research_agent/nodes/execute_task.jinja"
prompt_template = blob_manager.read_blob_as_template(prompt_path)

In [67]:
# [入力例]

# ここでは BuildResearchPlanNode から出力された依頼事項（ManagedTask）を使用

from app.domain.enums import Priority, ManagedTaskStatus
from app.workflow.models.build_research_plan import ManagedTask, TaskType

managed_task = ManagedTask(
    title='最終アウトプット設計と提言フレーム',
    overview='レポートの構成と提言の形式を設計',
    objective='再現性の高いレポートのテンプレートと、現場で使える提言フレームを作成',
    research_scope='対象: これまでのタスクの成果を統合; アウトオブスコープ: 現場の組織での適用手順は別タスク',
    priority=Priority.HIGH,
    required_capabilities=[TaskType.SEARCH, TaskType.THINKING],
    id='5soYw',
    status=ManagedTaskStatus.NOT_STARTED,
    deliverable=None,
    created_at='2025-10-30 05:05:52',
    updated_at='2025-10-30 05:05:52'
)

In [68]:
prompt = prompt_template.render(
    goal="日本市場におけるBPOの将来領域と新しいアウトソーシングビジネスモデル（成果報酬型、サブスクリプション、ハイブリッド）の実現可能性と価値提案を、将来展望の観点で総合的に評価するためのリサーチ計画を策定する。ユーザーの潜在的なニーズを推測して補完し、現実的に実行可能な道筋を提示する。最終的には、個別タスクが独立して並行実行可能な計画書を提供する。",
    title=managed_task.title,
    overview=managed_task.overview,
    objective=managed_task.objective,
    research_scope=managed_task.research_scope,
)

In [69]:
# プロンプトの確認

print(prompt)

あなたは優秀なリサーチアシスタントです。
このプロジェクトでは最終的に、採用プロセスにおけるAIエージェント活用の調査レポートをまとめることを目的としています。
本タスクではその一部として、特定の観点に絞った調査サブタスクを行います。

## あなたのタスク
与えられた観点について必要な情報を分かりやすく整理・調査し、調査結果と今後深掘りすべき点を丁寧に報告してください。
具体的には次のタスクを解いてください。

```toml
[最終目的]
日本市場におけるBPOの将来領域と新しいアウトソーシングビジネスモデル（成果報酬型、サブスクリプション、ハイブリッド）の実現可能性と価値提案を、将来展望の観点で総合的に評価するためのリサーチ計画を策定する。ユーザーの潜在的なニーズを推測して補完し、現実的に実行可能な道筋を提示する。最終的には、個別タスクが独立して並行実行可能な計画書を提供する。

[本タスク名]
最終アウトプット設計と提言フレーム

[タスク概要]
レポートの構成と提言の形式を設計

[タスクの目的]
再現性の高いレポートのテンプレートと、現場で使える提言フレームを作成

[調査範囲・スコープ]
対象: これまでのタスクの成果を統合; アウトオブスコープ: 現場の組織での適用手順は別タスク
```

### 使用可能なツール
以下のツールを使用できます：
1. search_web: Web検索を行うツール。特定のキーワードや観点について情報を検索します。複数回呼び出すことができます。
2. submit_content: 調査結果を提出するツール。マークダウン形式の提出物を作成した後、このツールを呼び出します。


### 実行手順

#### ステップ1: タスクの理解
  - 与えられた観点・目的・スコープを確認し、何を調査すべきかを明確にしてください。

#### ステップ2: 必要に応じたWeb検索による情報収集
  - タスクの性質を判断してください。単純な推論タスクや既存知識で対応可能なタスクの場合、このステップをスキップしても構いません。
  - Web検索が必要と判断した場合は、search_webツールを使用して、複数のキーワードで検索を行い、必要な情報を収集してください。
  - 必要に応じて複数回検索を実施し、異なる観点から情報を集めて

### 🤖 ReAct エージェント

In [70]:
import weave
weave.init(os.environ["PROJECT_NAME"])

weave: wandb version 0.22.3 is available!  To upgrade, please run:
weave:  $ pip install wandb --upgrade
weave: Logged in as Weights & Biases user: shumpei_miyawaki.
weave: View Weave data at https://wandb.ai/shumpei_miyawaki/wandb-ws-2025-ai-agent/weave


In [76]:
from langchain.agents import create_agent

agent = create_agent(
    model=f"openai:gpt-5-nano",
    tools=[search_web, submit_content],
    system_prompt=prompt,
).with_config({"recursion_limit": 50})

### 🏃‍♀️ 実行してみる

In [77]:
response = agent.invoke({})

weave: 🍩 https://wandb.ai/shumpei_miyawaki/wandb-ws-2025-ai-agent/r/call/019a32ac-f6b9-7960-a96e-887a2e3ecfa4


In [78]:
for message in response["messages"]:
    print(message.model_dump())

{'content': '', 'additional_kwargs': {'refusal': None}, 'response_metadata': {'token_usage': {'completion_tokens': 4939, 'prompt_tokens': 1747, 'total_tokens': 6686, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 1920, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1536}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CWBVrm6Ch8Skno9fPKGIeWvNDJUYS', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, 'type': 'ai', 'name': None, 'id': 'lc_run--9221bcb9-a9c2-43e6-a966-3dc4962b4b5a-0', 'tool_calls': [{'name': 'submit_content', 'args': {'content': '# 最終アウトプット設計と提言フレーム\n\nこのドキュメントは、日本市場におけるBPOの将来領域と新しいアウトソーシングビジネスモデル（成果報酬型、サブスクリプション、ハイブリッド）の実現可能性と価値提案を、将来展望の観点で総合的に評価するリサーチ計画のための、再現性の高いテンプレートと現場で使える提言フレームを提供する。各セクションは、個別タスクが独立して並行実行可能な形で設計されている。\n\n---\n\n## 1. レポート構成テンプレート（標準構成）\nこのテンプレートは、実データが不足してい

### ☕️ どう使い分ければ良い？


<img src="https://i.gyazo.com/5f4b8cd7100118a89374ceb083a65a89.png" />

## 🚩 Step3. エージェントの品質担保


### 導入
時間の都合上、割愛🙏

#### なぜ評価・テストする必要があるか？

評価やテストは開発初期のバグを防ぐだけでなく、リリース後も社会的なリスクや性能の劣化がないかを継続的にチェックするために必要となります。

1. **テスト：基礎品質の保証**<br/>
  システムが **仕様通りに** 正しく動くかをチェックします。また、システムをアップデートした際に、 **以前の機能が壊れていないか（改悪）** を防ぐための、最も基本的な品質チェックです。

2. **評価：性能向上と改善の羅針盤**<br/>
  AIの能力を測り、**改善点を見つける** ための分析です。特定のベンチマークと比較して自社のAIがどれだけ優れているかを確認したり、「どんな状況で得意で、どんな状況が苦手か」というAIの特性を深く理解し、次の改善につなげるために行います。

#### どの順番で着手すれば良いの？


<img src="https://i.gyazo.com/77ac427d299933829418d5f4acdbc339.png" />

#### なかでも生成タスクの評価は難しい

- 抽出や分類タスクであれば人が見て判断できる場合が多い。**ただし生成タスクは正解の判定基準が曖昧である。**
- レポートの受け入れ基準が存在すれば評価基準として設定する。**ただし決まっていないことが多い。**
- 実際の受け入れ実績のデータがあれば評価データとして使用する。**ただしコールドスタートである場合が多い。**

### ☕️ よくない評価の例

In [79]:
import asyncio
from concurrent.futures import ThreadPoolExecutor

import numpy as np
from openai import AsyncOpenAI
from pydantic import BaseModel, Field
from tqdm.asyncio import tqdm

client = OpenAI()


class UsefulnessScore(BaseModel):
    evaluation_result: str = Field(title="応答文の評価")
    score: int = Field(title="応答文の有益さ", gt=1, le=10)


def eval_usefulness_score(_=None):
    response = client.responses.parse(
        model="gpt-5-nano",
        input=[
            {"role": "system", "content": "アシスタントの応答文を『有益さ』の観点から1~10点で評価して。"},
            {"role": "user", "content": "こんにちは！"},
            {"role": "assistant", "content": "こんにちは！私はChatGPTです。何かお手伝いできることはありますか？（例：質問、翻訳、文章作成、学習支援、プログラミングなど）"},
        ],
        reasoning={
            "effort": "minimal"
        },
        text={
            "verbosity": "low"
        },
        text_format=UsefulnessScore,
    )
    usefulness_score = response.output_parsed
    return usefulness_score.score

#### なぜ良くないか？

In [81]:
with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_score = executor.map(eval_usefulness_score, [None] * 10)
    scores = list(tqdm(future_to_score, total=10, desc="running..."))

weave: 🍩 https://wandb.ai/shumpei_miyawaki/wandb-ws-2025-ai-agent/r/call/019a32af-6377-75b8-a01c-1872f3f26216
running...:   0%|          | 0/10 [00:00<?, ?it/s]weave: 🍩 https://wandb.ai/shumpei_miyawaki/wandb-ws-2025-ai-agent/r/call/019a32af-638a-7659-9463-9b8f3621ec5e
weave: 🍩 https://wandb.ai/shumpei_miyawaki/wandb-ws-2025-ai-agent/r/call/019a32af-6385-755a-bd36-d8a9dc6cdfdb
weave: 🍩 https://wandb.ai/shumpei_miyawaki/wandb-ws-2025-ai-agent/r/call/019a32af-6384-754c-b607-21ce972c2741
weave: 🍩 https://wandb.ai/shumpei_miyawaki/wandb-ws-2025-ai-agent/r/call/019a32af-6395-705b-92eb-afe7f689a412
weave: 🍩 https://wandb.ai/shumpei_miyawaki/wandb-ws-2025-ai-agent/r/call/019a32af-6397-769d-a9ac-dac2229f65ab
weave: 🍩 https://wandb.ai/shumpei_miyawaki/wandb-ws-2025-ai-agent/r/call/019a32af-6396-7ecf-9b20-6c8d473b4fc3
weave: 🍩 https://wandb.ai/shumpei_miyawaki/wandb-ws-2025-ai-agent/r/call/019a32af-6392-70ef-9387-4fcf26d34781
weave: 🍩 https://wandb.ai/shumpei_miyawaki/wandb-ws-2025-ai-agent/r/ca

In [83]:
# [確認] 実行ごとに評価の値にバラツキがある
print(f"{np.mean(scores):.04f} ± {np.std(scores):.04f}")
print(scores)

6.1000 ± 2.1656
[6, 2, 7, 10, 7, 7, 5, 7, 3, 7]


#### ばらつきが発生する原因は？

- 参加者の方々に質問

<!--
- 「有益さ」に対して多様な解釈が発生する
- 10段階評価ではあるが「どの場合に何点つけるか」が不明瞭
-->

##### 良いプロンプトの条件とは

<img src="https://pbs.twimg.com/media/G3S_hmQa4AEfMNW?format=jpg&name=4096x4096" />

<img src="https://i.gyazo.com/722f8cfbb395082a1afdd66260ce5fab.png" />

### ▶️ `submit_content` を改善してみよう

#### おさらい（再掲）

##### 出力形式

In [85]:
class Submission(BaseModel):
    status: EvaluationStatus = Field(
        title="提出物の評価結果",
        description=(
            "提出物の評価は3つのパターンで分けます。"
            "1. 'accepted'（受理）: 要件・期待値をすべて満たしており受け入れ可能。"
            "2. 'improvable'（改善の余地あり）: いくつか要件は満たしているが改善すべき点があり修正を推奨。"
            "3. 'rejected'（終了）: 主な要件が満たされておらず今回は受理不可。"
            "このいずれかを厳密に記入してください。"
        ),
    )
    reason: str = Field(
        title="受け入れ判断の詳細理由",
        description=(
            "提出物が受け入れられなかった場合は、どこが要件や期待に達していなかったのか、具体的な改善ポイントやアドバイスを丁寧に記述してください。"
            "逆に、受理された場合は、どの点が要件を満たしていたのか、なぜ十分だったのかについて積極的なフィードバックや称賛を含めて説明してください。"
        ),
        exclude=True,
    )

    @computed_field
    @property
    def reason_for_rejection(self) -> str | None:
        return None if self.status in [EvaluationStatus.REJECTED] else self.reason

##### プロンプト

In [7]:
! cat storage/prompts/research_agent/tools/submit_content.jinja

あなたは、提出物を丁寧かつ慎重に確認し、その受け入れ可否と根拠を論理的かつ建設的に提示する役割を担う、親切で誠実なアシスタントです。

## 評価方針

あなたのタスクは、以下の観点をもとに公平かつ専門的に提出物を審査し、分かりやすくフィードバックを行うことです。

### 評価基準 （status フィールドに従う）

提出物の評価は、以下の3つのいずれかの`status`値で厳密に分類してください。

#### 1. `accepted`（受理）
以下のすべての基準を満たしている場合、「accepted（受理）」と判定してください。
- **完全性**：タスクで求められた全要素や観点が網羅されている。
- **正確性**：事実誤認や矛盾がなく、内容が信頼できる。
- **構造**：情報が分かりやすく整理され、論理的かつ一貫性のある構成。
- **品質**：文体やフォーマットが専門的で明瞭、かつ必要な詳細さを備えている。
- **要件適合**：明示・暗黙の要件や期待値を十分に満たしている。

#### 2. `improvable`（改善の余地あり）
提出物に重大な致命的欠陥はないものの、いくつかの基準や細かな要件において不足や改善点が認められる場合、「improvable（改善の余地あり）」と判定してください。この場合、どの観点で不十分であり、どのように改善できるかを具体的にフィードバックしてください。
- **一部の観点の不足**：必須ほどではないが、追加・修正することでさらに良くなる要素がある。
- **軽微な不正確性や曖昧さ**：大幅な誤りではないが、記載内容の明確化や根拠の強化が望ましい箇所がある。
- **構成や品質面での小改善点**：論理展開は保たれているが、より整理・明確にできる部分がある。

#### 3. `rejected`（終了／受理不可）
提出物が下記いずれかに該当し、改善の余地がほぼない場合は「rejected（終了）」と判定してください。
- **極端な不完全性／逸脱**：必須の情報や観点がほとんど欠如、または全く別の内容である。
- **無関係または破綻した記述**：入力データが想定されるレポート形式や意図から大きく逸脱しており、内容が破綻・矛盾・混乱している。
- **改善が不可能なほど不適切**：根本的にタスク趣旨や要件に適合せ

- 現時点だと **AIエージェントの生成能力を知る術がない**
- AIエージェントの生成物に対する観測が `is_accepted` しかない
  - 評価基準（完全性、正確性など）に照らし合わせた評価値が観測できない
- 受理の可能性における最終的な判断をAIエージェントに依存しており、導出過程が不明瞭

#### 📈 改善方針を立てる

<img src="https://i.gyazo.com/25cda7d76412f816faf205c2a281cd19.png" />

In [93]:
# 有用性を例にしているが、本来は他の観点も追加されたい

evaluation_metrics = [
    {
        "title": "有用性",
        "description": "応答文がユーザーの目的や要求に対してどれだけ役立ち、価値を提供できるか（有益さ）。評価は、応答を利用して次の行動に移る際に必要な手間の少なさ（修正・補完の必要性）に基づき行う。",
        "scores": [
            {
                "value": 5,
                "description": "極めて有用: ユーザーの要求された情報/解決策を漏れなく提供し、期待を上回る実用的な洞察や革新的な提案を含む。応答をそのまま、またはコピペで即座に次の行動（実行、提示、利用）に活かすことができる。追加の調査や修正は一切不要。"
            },
            {
                "value": 4,
                "description": "非常に有用: ユーザーの要求を主要な点においてはすべて満たしている。提供された情報/解決策は正確かつ具体的であり、目的達成に直接貢献する。実用化には軽微な調整（例：形式の変更、用語の統一）のみが必要で、本質的な内容の補完や修正は不要。"
            },
            {
                "value": 3,
                "description": "標準的な有用性: ユーザーの要求の核となる部分は満たしているが、情報が一般的すぎる、または重要な要素の一部が欠けている。目的達成には、中程度の補完や調整（例：具体的な数値の追加、手順の明確化）が必要。"
            },
            {
                "value": 2,
                "description": "限定的な有用性: ユーザーの要求の表面的な部分のみに言及しており、解決策としては機能しない。情報が断片的、または誤解を招く可能性がある。目的達成のためには、応答の大幅な修正、再構築、または最初からのやり直しが必要。"
            },
            {
                "value": 1,
                "description": "有用性なし: ユーザーの要求に全く関連していない、または提供された情報が明確に間違っており、行動に悪影響を与える。利用価値は皆無。"
            }
        ]
    }
]

In [133]:
class HelpfulnessScore(BaseModel):
    reason: str = Field(title="採点理由")
    score: int = Field(title="採点スコア", gt=1, lt=5)


class Submission(BaseModel):
    metrics: list[HelpfulnessScore]
    reason: str = Field(
        title="受け入れ判断の詳細理由",
        description=(
            "提出物が受け入れられなかった場合は、どこが要件や期待に達していなかったのか、具体的な改善ポイントやアドバイスを丁寧に記述してください。"
            "逆に、受理された場合は、どの点が要件を満たしていたのか、なぜ十分だったのかについて積極的なフィードバックや称賛を含めて説明してください。"
        ),
        exclude=True,
    )

    @computed_field
    @property
    def reason_for_rejection(self) -> str | None:
        return None if self.status in [EvaluationStatus.REJECTED] else self.reason

    @computed_field
    @property
    def status(self) -> EvaluationStatus:
        # 受理可能かどうかの最終判断はコントロール可能にした
        if all(metric.score >= 4 for metric in self.metrics):
            return EvaluationStatus.ACCEPTED
        elif any(metric.score >= 2 for metric in self.metrics):
            return EvaluationStatus.IMPROVABLE
        else:
            return EvaluationStatus.REJECTED


In [134]:
%%writefile storage/prompts/research_agent/tools/submit_content.jinja
あなたは、提出物を丁寧かつ慎重に確認し、その受け入れ可否と根拠を論理的かつ建設的に提示する役割を担う、親切で誠実なアシスタントです。

## 評価方針

あなたのタスクは、以下の観点をもとに公平かつ専門的に提出物を審査し、分かりやすくフィードバックを行うことです。

### 評価観点

提出物に対して以下の観点でスコアづけをしてください。

```
<evaluation_metrics>
{% for metric in evaluation_metrics %}
<metric id="{{ metric.title }}">
  <description>{{ metric.description }}</description>
  <scores>{% for score in metric.scores %}
    <score>{{ score.value }}点: {{ score.description }}</score>
  {% endfor %}</scores>
</metric>
{% endfor %}
</evaluation_metrics>
```

## 出力形式

提出結果は、以下のJSONスキーマに厳密に従ってください。
なお metrics には evaluation_metrics に記述される全ての metric に対応するスコアを格納すること。
evaluation_metrics に含まれない観点については記述しないでください。

```jsonschema
{{ output_format | safe }}
```

- 判定理由（reason）は、受け入れ不可の場合は必ず詳細かつ建設的に記述し、どの基準を満たしていなかったのかも必ず明示してください。
- 受け入れ可の場合も、どの点が評価されたのか、優れていたポイントを分かりやすく具体的に称賛してください。

常に丁寧かつ客観的な態度で、提出物に対する実務的かつ誠実な評価結果と改善/称賛コメントをご提示ください。

Overwriting storage/prompts/research_agent/tools/submit_content.jinja


In [135]:
def submit_content(content: str) -> dict[str, str | bool | None]:
    """指定された提出物の内容を審査し、受け入れ可否および理由を返します。
    提出物はマークダウン形式で記述してください。本関数は、その内容が所定の要件や期待を満たしているかどうかを判定し、
    判定結果（受け入れ可否）および詳細な理由や改善点（または称賛ポイント）を返します。.

    Args:
    ----
        content (str): マークダウン形式で記述された確認対象の提出物の内容。

    Returns:
    -------
        Submission:
            is_accepted（bool）: 提出物が受け入れ可能かどうか（True: 受け入れ／False: 不可）。
            reason（str）: 受け入れ可否の根拠や詳細な理由。非受理の場合は改善ポイント、受理の場合は称賛コメントなどを含みます。
    """
    client = OpenAI()
    blob_manager = LocalBlobManager(log_level=LogLevel.TRACE)
    system_instruction_template = blob_manager.read_blob_as_template(
        "storage/prompts/research_agent/tools/submit_content.jinja"
    )
    system_instruction = system_instruction_template.render(
        output_format=Submission.model_json_schema(),
        evaluation_metrics=evaluation_metrics  # 追加
    )
    result: ParsedResponse[Submission] = client.responses.parse(
        model="gpt-5-nano",
        input=[
            {"role": "system", "content": system_instruction},
            {"role": "user", "content": content},
        ],
        text_format=Submission,
        reasoning={"effort": "low"},
        text={"verbosity": "low"},
    )
    submission: Submission = cast("Submission", result.output_parsed)
    return submission.model_dump()

In [136]:
# 試しに submit_content を実行してみる

content = open("storage/outputs/research_report.md").read()
submission = submit_content(content)
submission

weave: 🍩 https://wandb.ai/shumpei_miyawaki/wandb-ws-2025-ai-agent/r/call/019a331f-4139-72d8-aaea-ac24669733ad


{'metrics': [{'reason': '総体として本案は日本市場のHR/BPO領域に関する現状と5年ロードマップを網羅的に整理しており、初稿として高い有用性を持ちます。マイルストーン・ケースA–Fの要点、ビジネスモデル評価軸、KPI候補、出典案などが整理されており、今後の具体化・調整に直ちに着手可能です。特に「調査結果の要点」「ケーススタディと教訓」「今後深掘りすべき点」「初期ロードマップ案」など、実務設計へ落とし込むための骨格が整っている点を評価します。ただし、現時点での提出物としては以下の点を補足・整備すると、さらに実務適用性が高まります。',
   'score': 4}],
 'reason_for_rejection': '受け入れ可。ただし以下の改善ポイントを反映すれば、提案の実用性・再現性が大幅に高まります。\n- ケース別マトリクス表（A–F）の具体性の補完: 各ケースの要件、想定顧客セグメント、KPI、適用領域、価格設計、導入難易度を横断的に比較できる表形式の追加が望ましい。\n- 5年間ロードマップ案の具体性: 各Yearの具体的アクション項目、責任部署、KPIのマイルストーン、リスク対応の簡易表を追加すると実務設計に直結します。\n- 出典・データ検証の明示: 最新データに基づく定量根拠を強化するため、主要データソース（IDC Japan、矢野経済研究所等）の最新レポートリンクと参照日を本文中に挿入するか、別紙で整理してください。\n- 技術基盤・データガバナンスの具体化: HRIS/ATS/RPA連携、データ品質指標、セキュリティ要件、倫理・バイアス対策の実装案を、章立てで追加することを推奨します。\n- 出力形式の選択肢整理: マークダウン版以外にCSV/Excel指標表、ケース雛形の追加など、受取手の用途に合わせたファイル形式オプションを先に提示すると、納品物の受け渡しがスムーズです。\n\n次アクションとして、以下を選択ください。\n- ケース別マトリクス表を先に作成する案、または5年間ロードマップ案を先に作成する案、どちらを優先しますか？\n- 業界・企業規模の絞り込みリクエストの有無、特定の出典リスト付き版の優先度、提出形式の希望（マークダウン/CSV/Excel等）を教えてください。',
 'status': <Eval